In [ ]:
# !pip install sqlalchemy==1.4.7
# !pip install ibm_db_sa
# !pip install ipython-sql

In [1]:
# found panadas_profiling in a Titanic machine learning article. Had to run in Python Data environment.
import pandas as pd
from pandas_profiling import ProfileReport
from sqlalchemy import create_engine
from sqlalchemy import inspect

In [ ]:
# Load SQL using ipython-sql
# %load_ext sql

In [ ]:
# import os

In [2]:
host = "localhost"
database = "Titanic_Combined_sql"
user = "postgres"
password = "@Princess21"

In [ ]:
# connection_string = f"postgresql://{user}:{password}@{host}/{database}"

In [ ]:
# %sql $connection_string

In [ ]:
# %%sql 
# SELECT * from titanic_combined_sql LIMIT 10

In [3]:
# Import SQL into Jupyter - Module 8.5.1 code
db_string = f"postgresql://{user}:{password}@{host}/{database}"

In [4]:
engine = create_engine(db_string)

In [5]:
insp = inspect(engine)
insp.get_table_names()

['rms_titanic_passengers_and_crew',
 'rms_titanic_country_of_residency',
 'titanic',
 'titanic_survivors',
 'titanic_combined_sql',
 'filtered_data']

In [29]:
# Create a JOIN. Join 2 tables from SQL
titanic_combined = 'SELECT * FROM rms_titanic_passengers_and_crew \
                    LEFT JOIN rms_titanic_country_of_residency ON r.mainID = r.mainID)
engine(titanic_combined) 
result_set = engine.fetchall()
result_set.head()


SyntaxError: EOL while scanning string literal (<ipython-input-29-b1600e2b512b>, line 3)

In [30]:
# Import the data
df_titanic = pd.read_sql('SELECT * FROM titanic_combined_sql', engine)
df_titanic.head()

,index,passenger_name,born,died,age,age_at_death,gender,marital_status,category,ticket_class,...,job,pclass,survived,sibsp,parch,fare,boat,body,country_id2,survived_y_n
0,1000,Cohen Gurshon,1892.0,1978.0,19.0,85.0,Male,Single,Passenger,3rd Class,...,Printer / Compositor,None,NaN,NaN,NaN,NaN,None,None,359,1.0
1,1005,Colgan Joseph,1879.0,NaN,33.0,NaN,Male,None,Crew,Victualling,...,Scullion,None,NaN,NaN,NaN,NaN,None,None,363,1.0
2,1007,Collett Sidney Clarence Stuart,1887.0,1941.0,25.0,53.0,Male,Single,Passenger,2nd Class,...,None,None,NaN,NaN,NaN,NaN,None,None,364,1.0
3,1009,Collins John,1874.0,1950.0,38.0,76.0,Male,Married,Crew,Engine,...,Fireman,None,NaN,NaN,NaN,NaN,None,None,365,1.0
4,1010,Collins John,1894.0,1941.0,17.0,46.0,Male,Single,Crew,Victualling,...,None,None,NaN,NaN,NaN,NaN,None,None,None,1.0


In [31]:
# describe the data set
df_titanic.describe()

,born,died,age,age_at_death,survived,sibsp,parch,fare,survived_y_n
count,2885.000000,2453.000000,2885.000000,2454.000000,639.000000,639.000000,639.000000,639.000000,2911.000000
mean,1880.548700,1927.196494,30.963258,46.090465,0.483568,0.593114,0.525822,43.549655,0.244589
std,12.018167,22.436764,12.029410,23.531453,0.500121,0.930340,0.917715,58.804085,0.429917
min,1837.000000,1912.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1873.000000,1912.000000,23.000000,26.250000,0.000000,0.000000,0.000000,10.170850,0.000000
50%,1882.000000,1912.000000,29.000000,39.000000,0.000000,0.000000,0.000000,26.000000,0.000000
75%,1889.000000,1943.000000,38.000000,67.000000,1.000000,1.000000,1.000000,52.000000,0.000000
max,1912.000000,2009.000000,74.000000,104.000000,1.000000,5.000000,6.000000,512.329200,1.000000


In [ ]:
# Learned and read from internet reading up on the Kaggle titanic project about Exploratory Data Analysis. This is using the panadas-profiling library.
eda_report = ProfileReport(df_titanic)
eda_report

In [ ]:
# Initial imports. This code is directly from Module 17.
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
# trying plt.subplot found in Titanc literature and verified on matplotlib - the base structure that is
fig, ax = plt.subplots()
count = df_titanic['gender'].value_counts(sort=False)
survive_count = df_titanic.loc[df_titanic['survived'] == 1, ['gender']].value_counts(sort=False)
ax.bar('Female', survive_count['Female']/count['Female'], color='purple')
ax.bar('Male', survive_count['Male']/count['Male'], color='blue')
ax.set_ylabel('% Survived')

In [ ]:
# Encode the dataset - From Module 17
titanic_encoded = pd.get_dummies(df_titanic, columns=["Sex", "Embarked"])
titanic_encoded.head()

In [ ]:
# count the empty values
titanic_encoded.isna().sum()

In [ ]:
# Drop columns that missing rows or because it doesn't work 
titanic_encoded = titanic_encoded.drop(['Name', 'Ticket', 'Cabin'], axis = 1)

In [ ]:
# Remove rows missing values
titanic_encoded = titanic_encoded.dropna(subset = ['Age'])

In [ ]:
titanic_encoded.dtypes

In [ ]:
# Define the features set. From Module 17.8
X = titanic_encoded.copy()
X = X.drop("Survived", axis=1)
X.head()

In [ ]:
# Define the target set. From Module 17.8.
y = titanic_encoded["Survived"].ravel()
y[:5]

In [ ]:
# Splitting into Train and Test sets. From Module 17.8.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Creating a StandardScaler instance. From Module 17.8.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Create a random forest classifier. From Module 17.8.
rf_model = RandomForestClassifier(n_estimators=10, random_state=0) 

In [ ]:
# Fitting the model. From Module 17.8.
rf_model = rf_model.fit(X_train_scaled, y_train)

In [ ]:
# Making predictions using the testing data. From Module 17.8.
predictions = rf_model.predict(X_test_scaled)

In [ ]:
# Calculating the confusion matrix. From Module 17.8.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

In [ ]:
# Calculating the accuracy score. From Module 17.8.
acc_score = accuracy_score(y_test, predictions)

In [ ]:
# Displaying results. From Module 17.8.
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

In [ ]:
# Calculate feature importance in the Random Forest model. From Module 17.8.
importances = rf_model.feature_importances_
importances

In [ ]:
# We can sort the features by their importance. From Module 17.8.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

In [ ]:
importances.plot.bar()